# Snap Finance Analysis

The following is the code I used for the analysis I did for the BI engineer application for Snap Finance. For tasks 1-3, I just have sql queries to answer the given questions and get data for the visualizations.

For task 4, I used advanced machine learning techniques (SBERT and clustering) to analyze approval rate and amounts approved by job category, with the job categories being determined by semantic similarity as determined by a transformer encoder model, and the names of the clusters being determined by a decoder LLM (large language model).

All visualizations can be found at this Looker project: https://lookerstudio.google.com/reporting/166d3a1e-37cb-43c2-a786-a4b5455dd8a8

# Setup

In [ ]:
!pip install langchain_openai langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 2.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import duckdb
import matplotlib.pyplot as plt
import datasets
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
import torch
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
from dotenv import load_dotenv
import os

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [4]:
customers = pd.read_excel("sample_datasets.xlsx", sheet_name='customers')
applications = pd.read_excel("sample_datasets.xlsx", sheet_name='applications')
stores = pd.read_excel("sample_datasets.xlsx", sheet_name='stores')
marketing = pd.read_excel("sample_datasets.xlsx", sheet_name='marketing')

In [ ]:
load_dotenv('env')
API_KEY = os.getenv('OPENAI_API_KEY')

# Task 1

In [ ]:
# Check for duplicates
query = """
SELECT COUNT(application_id) AS num_ids
FROM applications
GROUP BY application_id
HAVING COUNT(application_id) > 1
"""

duckdb.query(query).to_df()

,num_ids


In [ ]:
# Number of applications
query = """
SELECT COUNT(application_id) AS number_of_apps_wo_duplicates
FROM applications
"""

duckdb.query(query).to_df()

,number_of_apps_wo_duplicates
0,65535


In [ ]:
# Number of approved applications
query = """
SELECT COUNT(application_id) AS number_of_approved_apps
FROM applications
WHERE approved = TRUE
"""

duckdb.query(query).to_df()

,number_of_approved_apps
0,32756


In [ ]:
# Number of used applications
query = """
SELECT COUNT(application_id) AS number_of_used_apps
FROM applications
WHERE dollars_used > 0
"""

duckdb.query(query).to_df()

,number_of_used_apps
0,16274


In [ ]:
# data for visualization
query = """
SELECT
  DATE_TRUNC('month', submit_date::DATE) AS month_bin,
  COUNT(application_id) AS total_apps,
  COUNT(CASE WHEN approved = TRUE THEN 1 END) AS approved_apps,
  COUNT(CASE WHEN dollars_used > 0 THEN 1 END) AS used_apps
FROM applications
GROUP BY month_bin
ORDER BY month_bin
"""

result = duckdb.query(query).to_df()
print(result)

    month_bin  total_apps  approved_apps  used_apps
0  2022-01-01        2775           1383        660
1  2022-02-01        2514           1282        628
2  2022-03-01        2723           1352        686
3  2022-04-01        2693           1284        649
4  2022-05-01        2787           1346        672
5  2022-06-01        2747           1391        712
6  2022-07-01        2824           1376        678
7  2022-08-01        2744           1379        683
8  2022-09-01        2706           1350        664
9  2022-10-01        2818           1422        694
10 2022-11-01        2720           1391        698
11 2022-12-01        2730           1359        654
12 2023-01-01        2709           1389        681
13 2023-02-01        2492           1284        647
14 2023-03-01        2808           1355        662
15 2023-04-01        2666           1351        659
16 2023-05-01        2682           1353        706
17 2023-06-01        2742           1363        703
18 2023-07-0

In [ ]:
result.to_csv("month_binned_used_apps.csv", index=False)

# Task 2

In [ ]:
# Average of approved amount
query = """
SELECT
  AVG(approved_amount) AS avg_approved_amount,
  AVG(NULLIF(dollars_used, 0)) AS avg_dollars_used
FROM applications
"""

duckdb.query(query).to_df()

,avg_approved_amount,avg_dollars_used
0,3122.597539,1617.177645


In [ ]:
# data for visualization
query = """
SELECT
  AVG(approved_amount) AS avg_approved_amount,
  AVG(CASE WHEN dollars_used > 0 THEN dollars_used END) AS avg_dollars_used,
  DATE_TRUNC('month', submit_date::DATE) AS month_bin
FROM applications
GROUP BY month_bin
ORDER BY month_bin
"""

result = duckdb.query(query).to_df()
result

,avg_approved_amount,avg_dollars_used,month_bin
0,3122.247289,1633.737879,2022-01-01
1,3155.090484,1585.361465,2022-02-01
2,3111.555473,1610.338192,2022-03-01
3,3098.978193,1610.996918,2022-04-01
4,3052.092868,1625.928571,2022-05-01
5,3125.365205,1597.436798,2022-06-01
6,3164.232558,1632.600295,2022-07-01
7,3128.906454,1581.159590,2022-08-01
8,3125.450370,1650.415663,2022-09-01
9,3148.229255,1631.048991,2022-10-01


In [ ]:
result.to_csv("month_binned_avg_approved_and_used.csv", index=False)

# Task 3

In [ ]:
query = """
SELECT
  store,
  COUNT(application_id) AS number_of_applications,
  SUM(CASE WHEN approved = TRUE THEN 1 ELSE 0 END) AS number_of_approved_applications,
  SUM(CASE WHEN approved = TRUE THEN 1 ELSE 0 END)/ COUNT(application_id) AS approval_rate,
  SUM(approved_amount) AS total_amount_approved,
  SUM(CASE WHEN dollars_used > 0 THEN 1 ELSE 0 END) AS number_of_used_applications,
  SUM(dollars_used) AS used_amount,
  SUM(dollars_used)/ SUM(approved_amount) AS usage_rate
FROM applications
GROUP BY store
ORDER BY store
"""

table = duckdb.query(query).to_df()
table

,store,number_of_applications,number_of_approved_applications,approval_rate,total_amount_approved,number_of_used_applications,used_amount,usage_rate
0,store_0,284,146.0,0.514085,448420.0,69.0,107970.0,0.240779
1,store_1,277,136.0,0.490975,431736.0,70.0,114715.0,0.265706
2,store_10,253,128.0,0.505929,388397.0,60.0,94609.0,0.243588
3,store_100,257,134.0,0.521401,431148.0,63.0,100645.0,0.233435
4,store_101,257,130.0,0.505837,405727.0,63.0,107608.0,0.265223
5,store_102,273,136.0,0.498168,404044.0,58.0,101340.0,0.250814
6,store_103,239,129.0,0.539749,405962.0,57.0,101780.0,0.250713
7,store_104,253,134.0,0.529644,401980.0,61.0,100515.0,0.250050
8,store_105,286,145.0,0.506993,447852.0,70.0,100685.0,0.224818
9,store_106,230,113.0,0.491304,354825.0,54.0,85418.0,0.240733


In [ ]:
table.to_csv("table_of_stats.csv", index=False)

# Task 4

In [31]:
query = """
CREATE OR REPLACE VIEW marketing_comparison AS
WITH application_data AS (
  SELECT
    a.dollars_used AS dollars_used,
    c.campaign AS campaign
  FROM applications a
  LEFT JOIN customers c ON a.customer_id = c.customer_id
)

SELECT
  SUM(COALESCE(a.dollars_used, 0)) AS dollars_used,
  MAX(m.spend) AS marketing_spend,
  m.name AS campaign_name
FROM marketing m
LEFT JOIN application_data a ON m.id = a.campaign
GROUP BY m.name
"""

duckdb.query(query)

In [32]:
query = """
SELECT * FROM marketing_comparison
"""
data = duckdb.query(query).to_df()
data

,dollars_used,marketing_spend,campaign_name
0,1202427.0,2728930,Content Marketing
1,1170408.0,1042189,Instagram Ads
2,1199160.0,704391,Facebook Ads
3,1111969.0,1944841,Email Marketing
4,1281597.0,3344380,Trade Shows
5,1201689.0,394522,Referral Marketing
6,1226074.0,3996752,Twitter Ads
7,1174320.0,4708501,Affiliate Marketing
8,9669848.0,0,No Campaign
9,1121866.0,2703354,SEO


In [37]:
query = """
SELECT SUM(marketing_spend) FROM marketing_comparison
"""
data = duckdb.query(query).to_df()
data

,sum(marketing_spend)
0,37612585.0


In [38]:
query = """
SELECT SUM(spend) FROM marketing
"""
data = duckdb.query(query).to_df()
data

,sum(spend)
0,37612585.0


In [39]:
query = """
SELECT SUM(dollars_used) FROM marketing_comparison
"""
data = duckdb.query(query).to_df()
data

,sum(dollars_used)
0,26317949.0


In [40]:
query = """
SELECT SUM(dollars_used) FROM applications
"""
data = duckdb.query(query).to_df()
data

,sum(dollars_used)
0,26317949.0


In [30]:
data.to_csv("dollars_used_and_marketing_spend.csv", index=False)

# Additional Analysis

In [41]:
query = """
SELECT
  c.title AS job_title,
  SUM(CASE WHEN a.approved = TRUE THEN 1 ELSE 0 END) AS num_approved,
  COUNT(a.application_id) AS total_applications,
  SUM(a.approved_amount) AS total_approved_amount,
  SUM(a.dollars_used) AS total_dollars_used
FROM customers c
INNER JOIN applications a ON c.customer_id = a.customer_id
GROUP BY job_title
"""

job_data = duckdb.query(query).to_df()
job_data

,job_title,num_approved,total_applications,total_approved_amount,total_dollars_used
0,Ambulance person,54.0,99,166384.0,45111.0
1,Social researcher,59.0,120,185009.0,27332.0
2,Illustrator,54.0,96,160876.0,44219.0
3,Surgeon,52.0,109,172693.0,27251.0
4,Tour manager,53.0,109,165434.0,44688.0
5,Occupational hygienist,57.0,119,176132.0,42232.0
6,"Geneticist, molecular",59.0,113,198820.0,33586.0
7,Multimedia programmer,62.0,110,194278.0,46908.0
8,Medical sales representative,51.0,112,169939.0,42650.0
9,Materials engineer,57.0,110,184753.0,40648.0


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

job_titles = job_data['job_title']

embeddings = model.encode(job_titles)

print(embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[-0.04196014  0.05815429  0.00962393 ... -0.10342854  0.04737473
   0.04033917]
 [-0.02859748 -0.03102986  0.02299123 ... -0.01134242  0.05703864
  -0.08009887]
 [-0.0361595   0.01741917 -0.01484122 ... -0.05758541 -0.00047317
   0.045883  ]
 ...
 [ 0.05146793  0.04366231 -0.03986334 ...  0.03930373  0.00739731
  -0.01485727]
 [-0.05327034 -0.00369993 -0.0551875  ...  0.02696963 -0.00378166
   0.01047334]
 [-0.0663733   0.0381097   0.02100852 ... -0.04442127  0.1372597
   0.05642587]]


In [ ]:
print(embeddings.shape)

(639, 384)


In [ ]:
normalized_embeddings = normalize(embeddings)

kmeans = KMeans(n_clusters=10, random_state=42)
job_data['cluster'] = kmeans.fit_predict(normalized_embeddings)

In [ ]:
job_data[['job_title', 'cluster']]

,job_title,cluster
0,IT sales professional,9
1,Licensed conveyancer,1
2,Archivist,3
3,Financial manager,1
4,Data scientist,5
5,"Administrator, charities/voluntary organisations",3
6,"Psychologist, counselling",0
7,Tax adviser,1
8,Physiological scientist,5
9,"Lecturer, higher education",2


In [ ]:
job_data[job_data['cluster'] == 4]

,job_title,num_approved,total_applications,total_approved_amount,total_dollars_used,cluster
14,"Producer, radio",53.0,95,146662.0,35955.0,4
67,"Journalist, magazine",60.0,109,199429.0,51156.0,4
70,Television production assistant,59.0,115,201979.0,51613.0,4
92,Newspaper journalist,51.0,107,158943.0,47192.0,4
116,"Engineer, broadcasting (operations)",41.0,100,137342.0,38486.0,4
126,Broadcast engineer,52.0,111,155314.0,38353.0,4
139,Radio producer,53.0,100,163917.0,52978.0,4
172,Civil Service fast streamer,51.0,104,155863.0,40239.0,4
176,Radio broadcast assistant,44.0,83,145817.0,33304.0,4
190,"Journalist, newspaper",53.0,113,154200.0,51190.0,4


In [ ]:
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=API_KEY)

prompt = ChatPromptTemplate.from_template(
    "You are a talent acquisition expert. Below is a list of job titles from a single cluster. "
    "Provide ONE concise, overarching professional job category (2-4 words) that describes them all.\n\n"
    "Titles: {titles}\n\n"
    "Overarching Title:"
)

cluster_names = {}

for cluster_id in sorted(job_data['cluster'].unique()):
    sample_titles = job_data[job_data['cluster'] == cluster_id]['job_title'].unique()[:20]
    titles_str = ", ".join(sample_titles)

    chain = prompt | llm
    response = chain.invoke({"titles": titles_str})

    cluster_names[cluster_id] = response.content.strip()
    print(f"Cluster {cluster_id}: {cluster_names[cluster_id]}")

job_data['cluster_name'] = job_data['cluster'].map(cluster_names)

Cluster 0: Mental Health and Therapy Professionals
Cluster 1: Financial Services Professionals
Cluster 2: Education and Library Professionals
Cluster 3: Public Sector and Community Services
Cluster 4: Media Production and Journalism
Cluster 5: Scientific Research Professional
Cluster 6: Surveying and Land Development
Cluster 7: Engineering Professionals
Cluster 8: Operations Manager
Cluster 9: Creative and Technical Design Professionals


In [ ]:
grouped_job_data = job_data.groupby('cluster').agg(
    num_approved=('num_approved', 'sum'),
    total_applications=('total_applications', 'sum'),
    total_approved_amount=('total_approved_amount', 'sum'),
    total_dollars_used=('total_dollars_used', 'sum'),
    cluster_name=('cluster_name', 'first'),
)

In [ ]:
grouped_job_data.head(10)

,num_approved,total_applications,total_approved_amount,total_dollars_used,cluster_name
cluster,,,,,
0,2401.0,4718,7500990.0,1932999.0,Mental Health and Therapy Professionals
1,4098.0,8171,12702012.0,3222723.0,Financial Services Professionals
2,1410.0,2821,4463436.0,1120448.0,Education and Library Professionals
3,5368.0,10633,16830516.0,4308014.0,Public Sector and Community Services
4,1733.0,3489,5357366.0,1359603.0,Media Production and Journalism
5,6616.0,13109,20647487.0,5264685.0,Scientific Research Professional
6,1173.0,2401,3680259.0,946580.0,Surveying and Land Development
7,3185.0,6364,9970570.0,2572410.0,Engineering Professionals
8,3363.0,6847,10514347.0,2831999.0,Operations Manager


In [ ]:
grouped_job_data = grouped_job_data.reset_index()

In [ ]:
query = """
SELECT
  cluster,
  num_approved,
  total_applications,
  num_approved/ total_applications AS approval_rate,
  total_approved_amount,
  total_dollars_used,
  total_approved_amount/ num_approved AS amount_per_application,
  total_dollars_used/ num_approved AS dollars_per_application,
  cluster_name
FROM grouped_job_data
ORDER BY cluster
"""

table = duckdb.query(query).to_df()
table

,cluster,num_approved,total_applications,approval_rate,total_approved_amount,total_dollars_used,amount_per_application,dollars_per_application,cluster_name
0,0,2401.0,4718,0.508902,7500990.0,1932999.0,3124.110787,805.080800,Mental Health and Therapy Professionals
1,1,4098.0,8171,0.501530,12702012.0,3222723.0,3099.563690,786.413616,Financial Services Professionals
2,2,1410.0,2821,0.499823,4463436.0,1120448.0,3165.557447,794.643972,Education and Library Professionals
3,3,5368.0,10633,0.504843,16830516.0,4308014.0,3135.342027,802.536140,Public Sector and Community Services
4,4,1733.0,3489,0.496704,5357366.0,1359603.0,3091.382574,784.537219,Media Production and Journalism
5,5,6616.0,13109,0.504691,20647487.0,5264685.0,3120.841445,795.750453,Scientific Research Professional
6,6,1173.0,2401,0.488546,3680259.0,946580.0,3137.475703,806.973572,Surveying and Land Development
7,7,3185.0,6364,0.500471,9970570.0,2572410.0,3130.477237,807.664050,Engineering Professionals
8,8,3363.0,6847,0.491164,10514347.0,2831999.0,3126.478442,842.104966,Operations Manager
9,9,3409.0,6982,0.488256,10616822.0,2758488.0,3114.350836,809.178058,Creative and Technical Design Professionals


In [ ]:
table.to_csv("bert_table.csv", index=False)